In [47]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [34]:
df = pd.read_excel('C:\Python\Capstone\Recommend_cars\Data\Data_Recommend.xlsx')
df.head(5)

,Tên Xe,Đơn giá,Người Bán,Địa chỉ người bán,Hãng xe,Năm SX,Dòng xe,Phiên bản,Kiểu dáng,Màu sắc,...,An toàn,Mức tiêu thụ nhiên liệu,Số loại sản phẩm,Thông tin chung,Khung xe,Tiện nghi,Ghế,An ninh,Truyền động - Hệ thống treo,Khoảng giá
0,"KIA Rondo KIA Rondo 2.0 GAT DELUXE 2016, Tự độ...",435000000,Bình Điền,Hồ Chí Minh,KIA,2016,RONDO,NaN,MPV,Bạc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 - 500 triệu
1,"Toyota Camry 2.5Q 2013, đã đi 160.000 km\n",580000000,Heesun,Hà Nội,TOYOTA,2013,CAMRY,2.5Q,Sedan,Xám,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500 triệu - 1 tỷ
2,Xe mới Ford Everest Wildtrak 2023,1499000000,Trần Huỳnh Thiết,Hồ Chí Minh,FORD,2023,EVEREST,NaN,SUV,Nhiều Màu,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 tỷ - 1.5 tỷ
3,"Toyota Rush Toyota Rush S 1.5AT 2020, Tự động,...",550000000,Viết Trung,Hồ Chí Minh,TOYOTA,2020,RUSH,NaN,SUV,Đen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500 triệu - 1 tỷ
4,"Mazda CX-5 SUV 2017, Tự động, đã đi 62.000 km",666000000,Đồng OTC,Hà Nội,MAZDA,2017,CX-5,NaN,SUV,Xám,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500 triệu - 1 tỷ


In [35]:
# Check for missing values in each column
missing_data = df.isnull().sum()
missing_data[missing_data > 0]

Dòng xe                          510
Phiên bản                      10425
Km đã đi                         526
Dung tích                       4752
Phân khúc                      14142
Động cơ - Mã lực               14142
Kích thước - Trọng lượng       14142
Động cơ - Hộp số               14142
Khung gầm                      14142
Ngoại thất                     14142
Nội thất                       14142
An toàn                        14142
Mức tiêu thụ nhiên liệu        14142
Số loại sản phẩm               14142
Thông tin chung                14142
Khung xe                       14142
Tiện nghi                      14142
Ghế                            14142
An ninh                        14142
Truyền động - Hệ thống treo    14142
dtype: int64

In [36]:
# Loại bỏ các dòng có giá trị khuyết thiếu trong các cột quan trọng
data_cleaned = df.dropna(subset=['Năm SX', 'Số chỗ ngồi', 'Km đã đi', 'Dung tích'])

# Điền giá trị khuyết thiếu cho các cột phân loại bằng giá trị xuất hiện nhiều nhất
for column in ['Dòng xe', 'Phiên bản', 'Kiểu dáng', 'Màu sắc', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']:
    most_common = data_cleaned[column].mode()[0]
    data_cleaned[column] = data_cleaned[column].fillna(most_common)

# Hiển thị thông tin mới của dữ liệu sau khi đã xử lý giá trị khuyết thiếu
data_cleaned.info(), data_cleaned.head()


<class 'pandas.core.frame.DataFrame'>
Index: 9390 entries, 580 to 10133
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Tên Xe                       9390 non-null   object 
 1   Đơn giá                      9390 non-null   int64  
 2   Người Bán                    9390 non-null   object 
 3   Địa chỉ người bán            9390 non-null   object 
 4   Hãng xe                      9390 non-null   object 
 5   Năm SX                       9390 non-null   int64  
 6   Dòng xe                      9390 non-null   object 
 7   Phiên bản                    9390 non-null   object 
 8   Kiểu dáng                    9390 non-null   object 
 9   Màu sắc                      9390 non-null   object 
 10  Tình trạng                   9390 non-null   object 
 11  Số chỗ ngồi                  9390 non-null   int64  
 12  Xuất xứ                      9390 non-null   object 
 13  Tỉnh thành          

C:\Users\Asus\AppData\Local\Temp\ipykernel_20524\3682035959.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[column] = data_cleaned[column].fillna(most_common)


(None,
                       Tên Xe    Đơn giá            Người Bán  \
 580  Audi A1 1.4 TFSI - 2010  368000000      Good Car Hà Nội   
 581    Audi A3 1.4 AT - 2011  538000000  Salon Auto Tân Tiến   
 582      Audi A4 1.8T - 2011  365000000              Mr Phúc   
 583  Audi A4 1.8 TFSI - 2015  600000000           Manh Duong   
 584  Audi A4 2.0 TFSI - 2017  799000000   Ngọc Quỳnh Autocar   
 
                                      Địa chỉ người bán Hãng xe  Năm SX  \
 580                     168 Phú Thượng - Tây Hồ Hà Nội    AUDI    2010   
 581  244 Phố Tây Sơn - Thị trấn Phùng - Đan Phượng ...    AUDI    2011   
 582           Hạ Yên Quyết - Yên Hoà - Cầu Giấy Hà Nội    AUDI    2011   
 583         415 Trường Trinh - P14 - Q Tân Bình TP HCM    AUDI    2015   
 584             439 Cộng Hòa - P15 - Q Tân Bình TP HCM    AUDI    2017   
 
     Dòng xe Phiên bản  Kiểu dáng Màu sắc  ... An toàn  \
 580      A1   1.5 AT   Hatchback      Đỏ  ...     NaN   
 581      A3   1.5 AT   Hatchback

In [39]:
# Chọn các cột phân loại để mã hóa
categorical_features = ['Kiểu dáng', 'Màu sắc', 'Hộp số', 'Nhiên liệu', 'Hãng xe']

# Khởi tạo OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' để tránh multicollinearity

# Fit và transform các cột phân loại
encoded_features = encoder.fit_transform(data_cleaned[categorical_features])

# Tạo DataFrame cho các cột đã mã hóa
encoded_features_df = pd.DataFrame(encoded_features, 
                                   columns=encoder.get_feature_names_out(categorical_features),
                                   index=data_cleaned.index)

# Ghép các cột đã mã hóa vào DataFrame gốc, loại bỏ các cột phân loại cũ
data_final = data_cleaned.drop(categorical_features, axis=1).join(encoded_features_df)

# Hiển thị thông tin và vài dòng đầu tiên của DataFrame cuối cùng
data_final.info(), data_final.head()

<class 'pandas.core.frame.DataFrame'>
Index: 9390 entries, 580 to 10133
Data columns (total 91 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Tên Xe                           9390 non-null   object 
 1   Đơn giá                          9390 non-null   int64  
 2   Người Bán                        9390 non-null   object 
 3   Địa chỉ người bán                9390 non-null   object 
 4   Năm SX                           9390 non-null   int64  
 5   Dòng xe                          9390 non-null   object 
 6   Phiên bản                        9390 non-null   object 
 7   Tình trạng                       9390 non-null   object 
 8   Số chỗ ngồi                      9390 non-null   int64  
 9   Xuất xứ                          9390 non-null   object 
 10  Tỉnh thành                       9390 non-null   object 
 11  Km đã đi                         9390 non-null   float64
 12  Link xe               

(None,
                       Tên Xe    Đơn giá            Người Bán  \
 580  Audi A1 1.4 TFSI - 2010  368000000      Good Car Hà Nội   
 581    Audi A3 1.4 AT - 2011  538000000  Salon Auto Tân Tiến   
 582      Audi A4 1.8T - 2011  365000000              Mr Phúc   
 583  Audi A4 1.8 TFSI - 2015  600000000           Manh Duong   
 584  Audi A4 2.0 TFSI - 2017  799000000   Ngọc Quỳnh Autocar   
 
                                      Địa chỉ người bán  Năm SX Dòng xe  \
 580                     168 Phú Thượng - Tây Hồ Hà Nội    2010      A1   
 581  244 Phố Tây Sơn - Thị trấn Phùng - Đan Phượng ...    2011      A3   
 582           Hạ Yên Quyết - Yên Hoà - Cầu Giấy Hà Nội    2011      A4   
 583         415 Trường Trinh - P14 - Q Tân Bình TP HCM    2015      A4   
 584             439 Cộng Hòa - P15 - Q Tân Bình TP HCM    2017      A4   
 
     Phiên bản Tình trạng  Số chỗ ngồi    Xuất xứ  ... Hãng xe_PORSCHE  \
 580   1.5 AT       Xe cũ            4  Nhập khẩu  ...             0.0   
 

In [40]:
# Tách dữ liệu thành features và target
X = data_final.drop(['Tên Xe', 'Đơn giá', 'Người Bán', 'Địa chỉ người bán', 'Dòng xe', 'Phiên bản', 'Tình trạng', 'Xuất xứ', 'Tỉnh thành', 'Link xe', 'Image_URL', 'Phân khúc',
       'Động cơ - Mã lực', 'Kích thước - Trọng lượng', 'Động cơ - Hộp số',
       'Khung gầm', 'Ngoại thất', 'Nội thất', 'An toàn',
       'Mức tiêu thụ nhiên liệu', 'Số loại sản phẩm', 'Thông tin chung',
       'Khung xe', 'Tiện nghi', 'Ghế', 'An ninh',
       'Truyền động - Hệ thống treo', 'Khoảng giá'], axis=1)
y = data_final['Đơn giá']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Kiểm tra kích thước của tập huấn luyện và tập kiểm tra
X_train_scaled.shape, X_test_scaled.shape

((7512, 63), (1878, 63))

In [43]:
# Huấn luyện mô hình KNN với số lượng láng giềng k=5
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# Dự đoán giá xe trên tập kiểm tra
y_pred = knn_model.predict(X_test_scaled)

# Tính toán các chỉ số đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2

(8.933029483280086e+17, 0.8224987940611821)

In [45]:
import joblib

# Lưu mô hình KNN
joblib.dump(knn_model, 'C:\Python\Capstone\Recommend_cars\knn_model2.joblib', protocol=4)

# Lưu bộ mã hóa OneHotEncoder
joblib.dump(encoder, 'C:\Python\Capstone\Recommend_cars\encoder2.joblib', protocol=4)

# Lưu bộ chuẩn hóa StandardScaler
joblib.dump(scaler, 'C:\Python\Capstone\Recommend_cars\scaler2.joblib', protocol=4)

# Trả về đường dẫn của các file đã lưu
'C:\Python\Capstone\Recommend_cars\knn_model1.joblib', 'C:\Python\Capstone\Recommend_cars\encoder1.joblib', 'C:\Python\Capstone\Recommend_cars\scaler1.joblib'

('C:\\Python\\Capstone\\Recommend_cars\\knn_model1.joblib',
 'C:\\Python\\Capstone\\Recommend_cars\\encoder1.joblib',
 'C:\\Python\\Capstone\\Recommend_cars\\scaler1.joblib')